# MERCS 101 - Lecture 01: Classification

This is the first part of the tutorial, focusing on MERCS as a simple classifier.

## Preliminaries

### External Imports

In [1]:
import numpy as np
import os
import sys
from sklearn.metrics import f1_score, accuracy_score, classification_report
import pandas as pd

### MERCS imports

In [2]:
sys.path.insert(0, '..') # We add the parent dir to the path
from src.mercs.core import MERCS
from src.mercs.utils import *

import src.datasets as datasets

/home/elia/Software/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


ImportError: cannot import name 'codes_to_query'

## Induction

### Importing Data

First, we import the nursery dataset.

In [ ]:
train, test = datasets.load_nursery()

This is a fully nominal dataset

In [ ]:
train.head()

### Training

In [ ]:
model = MERCS()

In [ ]:
ind_parameters = {'ind_type':           'RF',
                  'ind_n_estimators':   30}

sel_parameters = {'sel_type':           'Base',
                  'sel_its':            4,
                  'sel_param':          2}

In [ ]:
model.fit(train, **ind_parameters, **sel_parameters)

## Inference

### Prediction

In [ ]:
code = [0,0,0,0,0,0,0,0,1]
len(code)

In [ ]:
pred_parameters = {'pred_type':    'MI',
                   'pred_param':   1.0,
                   'pred_its':     8}

In [ ]:
y_pred = model.predict(test,
                       **pred_parameters,
                       qry_code=code)

In [ ]:
y_pred

### Evaluation 

In [ ]:
y_true = test[test.columns.values[np.array(code)==1]].values

In [ ]:
obs = f1_score(y_true, y_pred, average='macro')
obs

In [ ]:
assert isinstance(obs, (int, float))
assert 0 <= obs <= 1